In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed

In [2]:
data = pd.read_csv("data_60_points.csv")
final_df = pd.read_csv("real_trainable_data.csv")

In [3]:
final_df.head()

,0,1,2,3,4,5,6,7,8,9,...,471,472,473,474,475,476,477,478,479,Target
0,24.292663,19.482136,8.624346,-0.932498,-0.395269,-0.455120,0.855522,0.265530,0.903650,0.375534,...,-17.311551,-22.457486,-28.753222,-27.479641,-28.066095,-25.329087,-24.605612,-24.266738,-21.633146,1.0
1,13.989555,10.538844,4.357757,-4.671976,-2.448575,-0.195571,0.800786,3.143261,5.239468,4.201113,...,-17.004589,-22.251523,-24.471177,-27.359055,-32.696794,-36.146539,-31.180339,-26.265950,-24.452572,2.0
2,28.226338,22.926922,12.568386,5.334923,5.037145,4.942274,5.231862,5.857457,4.706730,3.209744,...,-23.205840,-26.171166,-28.149451,-30.068256,-28.035475,-23.902512,-21.659425,-22.133117,-22.457960,3.0
3,10.275999,4.947967,-2.816720,-0.592784,3.641866,4.649932,2.646541,-1.270290,-2.929141,-6.799285,...,-23.478831,-25.957031,-28.022580,-31.121322,-33.086501,-26.612358,-23.061996,-22.466169,-24.421495,4.0
4,25.701159,19.995006,9.928850,2.699057,3.008508,3.815365,4.283556,5.071418,6.786370,5.908505,...,-16.841160,-16.545065,-18.070230,-22.589697,-27.299603,-27.420140,-24.701882,-23.104943,-23.844575,5.0


In [26]:
inputs = final_df.drop("Target", axis = 1)
targets = final_df.Target 
targets = targets.apply(lambda x: int(x))

### Own Train and Validation Split

In [48]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(inputs, targets, test_size=0.15, random_state=42)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((20808, 480), (20808,), (3672, 480), (3672,))

### Normalization and Standarization 

In [49]:
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.pipeline import Pipeline

# configure our pipeline
pipeline = Pipeline([('normalizer', Normalizer()),
                     ('scaler', MinMaxScaler())])

In [50]:
#pipeline.fit(x_train)

#x_train = pipeline.transform(x_train)
#x_test = pipeline.transform(x_test)

### Creating Subsequences

In [51]:
# Subsequence for each 60 timestep (each a different channel)

def create_subsequences(x,timestep = 1):
    Xs = []
    stop_index = ((len(x)-timestep))
    
    for i in range(len(x)):
        if(i<=stop_index):
            Xs.append(x.iloc[i:(i+timestep),:])
        else:
            #same_array = [x[i,:]]*10
            Xs.append(x.iloc[(i-timestep):i,:])
    return np.array(Xs)

In [52]:
x_train = create_subsequences(x_train)
x_test = create_subsequences(x_test)

In [53]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((20808, 1, 480), (3672, 1, 480), (20808,), (3672,))

## LSTM MODEL

In [57]:
model = Sequential()

timesteps = x_train.shape[1]
nfeatures = x_train.shape[2]
input_shape = (timesteps,nfeatures)

outputs = len(y_train.unique())

model.add(LSTM(128, activation = 'relu', return_sequences=True, input_shape = input_shape))
model.add(Dropout(0.5))
model.add(LSTM(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(RepeatVector(timesteps))
model.add(LSTM(64, activation = 'relu',return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, activation = 'relu',return_sequences=True))
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(outputs, activation="softmax")))

model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_40 (LSTM)              (None, 1, 128)            311808    
                                                                 
 dropout_40 (Dropout)        (None, 1, 128)            0         
                                                                 
 lstm_41 (LSTM)              (None, 64)                49408     
                                                                 
 dropout_41 (Dropout)        (None, 64)                0         
                                                                 
 repeat_vector_10 (RepeatVec  (None, 1, 64)            0         
 tor)                                                            
                                                                 
 lstm_42 (LSTM)              (None, 1, 64)             33024     
                                                     

In [55]:
from keras.callbacks import ModelCheckpoint, TensorBoard

#model.compile(loss='mse', optimizer=adam, metrics=["acc"])
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0.0001,patience=3,verbose=1, mode='min',restore_best_weights=True)

history = model.fit(x_train, y_train, epochs=15, validation_data=(x_test, y_test),verbose=2, callbacks=[early_stop])

Epoch 1/15


ValueError: in user code:

    File "c:\Users\alexc\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\alexc\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\alexc\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\alexc\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 860, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\alexc\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 918, in compute_loss
        return self.compiled_loss(
    File "c:\Users\alexc\Ap